In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
from address_extractor import get_detailed_address

In [2]:
df_full = pd.read_csv(r'./flatforsaleindhaka_spider.csv')

In [3]:
df1 = df_full.copy()

In [4]:
df1.head(3)

,baths,beds,location,parking,price,price_per_sqft,property_type,property_url,sqft
0,NaN,NaN,"Dhaka, Motijheel, Old Town",NaN,"2,000,000 BDT","20,000.00 BDT",Commercial,https://flatforsaleindhaka.com/listings/ready-...,100
1,3,3,"Dhaka, Green Road",1.0,"6,900,000 BDT","6,000.00 BDT",Apartment,https://flatforsaleindhaka.com/listings/ready-...,"1,150"
2,2,2,"Dhaka, Baridhara, Bashundhara, Kalachadpur",1.0,"3,357,500 BDT","3,950.00 BDT",Apartment,https://flatforsaleindhaka.com/listings/%e0%a6...,850


In [5]:
df1.shape

(21, 9)

In [6]:
df_fin = pd.DataFrame()

In [ ]:
df_fin["city"] = np.NaN
df_fin["locality"] = np.NaN
df_fin["address"] = np.nan

In [7]:
# New code

"""
    Loop through `location` column, while splitting each location to city, zone, address and add them
        to the relevant column
"""

for index, row in df1.iterrows(): # loop through each sample
    
    # The code may take time, log in the console to keep track of things
    if index==0 or index%1000==0:
        print(f"Currently processing sample {index}...")
        
    # retrieve the location
    location = df1.loc[index, "location"]
    
    # split the location to dictionary with Area, City, Address as keys
    location_dict = get_detailed_address(location)
    print(location_dict)
    
    city = location_dict.get("city", np.NaN)
    locality = location_dict.get("area", np.NaN)
    address = location_dict.get("address", np.NaN)
    

    # updating the relevant columns of the sample in the dataframe
    df_fin.loc[index, "city"] = city
    df_fin.loc[index, "locality"] = locality
    df_fin.loc[index, "address"] = address

print("Processing has come to an end")

Currently processing sample 0...
{'city': 'Dhaka', 'area': 'Motijheel', 'address': ' Old Town '}
{'city': 'Dhaka', 'area': '', 'address': ' Green Road '}
{'city': 'Dhaka', 'area': 'Baridhara', 'address': ' Kalachadpur '}
{'city': 'Dhaka', 'area': 'Motijheel', 'address': ' Segun Bagicha'}
{'city': 'Dhaka', 'area': 'Gulshan', 'address': ''}
{'city': 'Dhaka', 'area': 'Dhanmondi', 'address': ' Old Town '}
{'city': 'Dhaka', 'area': 'Motijheel', 'address': ' Segun Bagicha'}
{'city': 'Dhaka', 'area': 'Shantinagar', 'address': ' Segun Bagicha'}
{'city': 'Dhaka', 'area': 'Dhanmondi', 'address': ''}
{'city': 'Dhaka', 'area': 'Mirpur', 'address': ''}
{'city': 'Dhaka', 'area': 'Dhanmondi', 'address': ''}
{'city': 'Dhaka', 'area': 'Khilgaon', 'address': ''}
{'city': 'Dhaka', 'area': 'Tejgaon', 'address': ''}
{'city': 'Dhaka', 'area': 'Dhanmondi', 'address': ''}
{'city': 'Dhaka', 'area': 'Bashundhara', 'address': ''}
{'city': 'Dhaka', 'area': 'Mohammadpur', 'address': ''}
{'city': 'Dhaka', 'area': '

In [8]:
#area - to be decimal and not string 
df_fin['area'] = df1.sqft.apply(lambda x: x.replace(',',''))
df_fin.area = df_fin.area.astype(float)
df_fin.area.dtypes

dtype('float64')

In [9]:
#building_type
df_fin['building_type']  = df1.property_type
df_fin.building_type.unique()

array(['Commercial', 'Apartment', 'Apartment, Commercial'], dtype=object)

In [10]:
df_fin.building_type = df_fin.building_type.replace('Commercial', 'Commerical - Other')
#df_fin.building_type = df_fin.building_type.replace('Apartment, Commercial', 'Commerical - Other')
df_fin.building_type.unique()

array(['Commerical - Other', 'Apartment', 'Apartment, Commercial'],
      dtype=object)

In [11]:
df1.property_type.unique()

array(['Commercial', 'Apartment', 'Apartment, Commercial'], dtype=object)

In [12]:
#building_nature
df_fin['building_nature'] = df1.property_type
df_fin.building_nature = df_fin.building_nature.replace('Apartment','Residential')
df_fin.building_nature.unique()

array(['Commercial', 'Residential', 'Apartment, Commercial'], dtype=object)

In [13]:
#num of bath and bed rooms
df_fin['num_bath_rooms'] = df1.baths
df_fin['num_bed_rooms'] = df1.beds

#fill na with 0
df_fin.num_bath_rooms.fillna(0,inplace=True)
df_fin.num_bed_rooms.fillna(0,inplace=True)

df_fin.num_bath_rooms.isna().sum(), df_fin.num_bed_rooms.isna().sum()

(0, 0)

In [14]:
#price
df_fin['price'] = df1.price.apply(lambda x: x.split(' BDT')[0].replace(',','') )
df_fin.price = df_fin.price.astype(float)
df_fin.price.dtype

dtype('float64')

In [15]:
#purpose
#since the data is not available, and the website mentions the sale of property, defaulting value to sale
df_fin['purpose'] = 'Sale'

In [16]:
#parking
df_fin['parking-spaces-amenity'] = df1.parking
df_fin['parking-spaces-amenity'].fillna(0,inplace=True)

In [17]:
df_fin['property_description'] = np.NaN
df_fin['property_overview'] = np.NAN

In [18]:
df_fin.head(6).T

,0,1,2,3,4,5
city,Dhaka,Dhaka,Dhaka,Dhaka,Dhaka,Dhaka
locality,Motijheel,,Baridhara,Motijheel,Gulshan,Dhanmondi
address,Old Town,Green Road,Kalachadpur,Segun Bagicha,,Old Town
area,100.0,1150.0,850.0,2650.0,7556.0,820.0
building_type,Commerical - Other,Apartment,Apartment,Apartment,Commerical - Other,"Apartment, Commercial"
building_nature,Commercial,Residential,Residential,Residential,Commercial,"Apartment, Commercial"
num_bath_rooms,0,3,2,"3, 4",0,0
num_bed_rooms,0,3,2,"3, 4",0,0
price,2000000.0,6900000.0,3357500.0,22525000.0,302240000.0,3280000.0
purpose,Sale,Sale,Sale,Sale,Sale,Sale


In [19]:
df_fin.shape

(21, 13)

In [20]:
df3 = df_fin.copy()
flatforsaleindhaka = (
    df3
    .assign(
        num_bath_rooms = lambda x: x["num_bath_rooms"].str.split(", "),
        num_bed_rooms = lambda x: x["num_bed_rooms"].str.split(", "),
        building_nature = lambda x: x["building_nature"].str.split(", ")
    )
    .explode("num_bath_rooms")
    .explode("num_bed_rooms")
    .explode("building_nature")
)

In [21]:
flatforsaleindhaka

,city,locality,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,parking-spaces-amenity,property_description,property_overview
0,Dhaka,Motijheel,Old Town,100.0,Commerical - Other,Commercial,NaN,NaN,2000000.0,Sale,0.0,NaN,NaN
1,Dhaka,,Green Road,1150.0,Apartment,Residential,3,3,6900000.0,Sale,1.0,NaN,NaN
2,Dhaka,Baridhara,Kalachadpur,850.0,Apartment,Residential,2,2,3357500.0,Sale,1.0,NaN,NaN
3,Dhaka,Motijheel,Segun Bagicha,2650.0,Apartment,Residential,3,3,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,Motijheel,Segun Bagicha,2650.0,Apartment,Residential,3,4,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,Motijheel,Segun Bagicha,2650.0,Apartment,Residential,4,3,22525000.0,Sale,1.0,NaN,NaN
3,Dhaka,Motijheel,Segun Bagicha,2650.0,Apartment,Residential,4,4,22525000.0,Sale,1.0,NaN,NaN
4,Dhaka,Gulshan,,7556.0,Commerical - Other,Commercial,NaN,NaN,302240000.0,Sale,2.0,NaN,NaN
5,Dhaka,Dhanmondi,Old Town,820.0,"Apartment, Commercial",Apartment,NaN,NaN,3280000.0,Sale,1.0,NaN,NaN
5,Dhaka,Dhanmondi,Old Town,820.0,"Apartment, Commercial",Commercial,NaN,NaN,3280000.0,Sale,1.0,NaN,NaN


In [22]:
flatforsaleindhaka.num_bath_rooms.fillna(0,inplace=True)
flatforsaleindhaka.num_bed_rooms.fillna(0,inplace=True)

In [23]:
flatforsaleindhaka.num_bath_rooms = flatforsaleindhaka.num_bath_rooms.astype(float)
flatforsaleindhaka.num_bed_rooms = flatforsaleindhaka.num_bed_rooms.astype(float)

In [24]:
flatforsaleindhaka.shape

(25, 13)

In [25]:
df_fin

,city,locality,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,parking-spaces-amenity,property_description,property_overview
0,Dhaka,Motijheel,Old Town,100.0,Commerical - Other,Commercial,0,0,2000000.0,Sale,0.0,NaN,NaN
1,Dhaka,,Green Road,1150.0,Apartment,Residential,3,3,6900000.0,Sale,1.0,NaN,NaN
2,Dhaka,Baridhara,Kalachadpur,850.0,Apartment,Residential,2,2,3357500.0,Sale,1.0,NaN,NaN
3,Dhaka,Motijheel,Segun Bagicha,2650.0,Apartment,Residential,"3, 4","3, 4",22525000.0,Sale,1.0,NaN,NaN
4,Dhaka,Gulshan,,7556.0,Commerical - Other,Commercial,0,0,302240000.0,Sale,2.0,NaN,NaN
5,Dhaka,Dhanmondi,Old Town,820.0,"Apartment, Commercial","Apartment, Commercial",0,0,3280000.0,Sale,1.0,NaN,NaN
6,Dhaka,Motijheel,Segun Bagicha,1150.0,Commerical - Other,Commercial,0,0,18400000.0,Sale,1.0,NaN,NaN
7,Dhaka,Shantinagar,Segun Bagicha,1590.0,Apartment,Residential,3,3,16000000.0,Sale,1.0,NaN,NaN
8,Dhaka,Dhanmondi,,91.0,Commerical - Other,Commercial,0,0,3640000.0,Sale,0.0,NaN,NaN
9,Dhaka,Mirpur,,1405.0,Apartment,Residential,3,3,6322500.0,Sale,1.0,NaN,NaN


In [26]:
flatforsaleindhaka.to_csv(r'./cleaned_flatforsaleindhaka_spider.csv',index=False)

In [28]:
flatforsaleindhaka.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 20
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    25 non-null     object 
 1   locality                25 non-null     object 
 2   address                 25 non-null     object 
 3   area                    25 non-null     float64
 4   building_type           25 non-null     object 
 5   building_nature         25 non-null     object 
 6   num_bath_rooms          25 non-null     float64
 7   num_bed_rooms           25 non-null     float64
 8   price                   25 non-null     float64
 9   purpose                 25 non-null     object 
 10  parking-spaces-amenity  25 non-null     float64
 11  property_description    0 non-null      float64
 12  property_overview       0 non-null      float64
dtypes: float64(7), object(6)
memory usage: 2.7+ KB
